In [1]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_30.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "30_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "30_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "30_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "30_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "30_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "30_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


submatrix 29_cross_matrix_1_2.csv has been saved
submatrix 29_cross_matrix_1_3.csv has been saved
submatrix 29_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,-0.902848,-0.920645,0.903978,0.911143,-0.874082,0.886079,0.90592,0.858526,-0.775081,-0.939825,...,-0.811917,-0.914729,-0.824972,-0.936681,-0.928033,0.880968,-0.960166,-0.909303,-0.869017,-0.782344
21085.0,-0.88879,-0.916905,0.897194,-0.902335,0.849351,-0.893922,-0.889059,0.865749,-0.80925,-0.906781,...,-0.691503,0.882871,-0.799279,-0.920372,-0.856618,-0.853047,-0.894307,-0.85752,-0.83318,-0.733351
21087.0,-0.877096,-0.925884,-0.890749,-0.905757,-0.812304,-0.913517,-0.922735,0.877051,-0.832653,-0.871776,...,-0.720669,-0.954969,-0.84946,-0.972277,-0.900955,-0.936777,-0.972113,-0.982459,-0.83993,-0.776274
21089.0,-0.868358,-0.900276,0.882269,0.893404,-0.912098,-0.915079,-0.898088,0.867663,-0.858193,-0.950286,...,-0.75348,-0.918901,-0.765003,-0.908949,-0.870582,-0.865443,-0.960261,-0.89934,-0.905189,-0.754922
21091.0,-0.836834,-0.871565,-0.889887,-0.879788,0.873866,-0.862939,0.89955,0.868145,0.870631,-0.868942,...,-0.644521,-0.868547,-0.742643,-0.871696,-0.775135,-0.944605,-0.905285,-0.855074,-0.861102,-0.736837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,-0.970917,0.970532,0.962433,0.95349,-0.958366,1.008495,1.011807,1.008216,0.907375,0.987174,...,-0.804138,0.920779,-0.812372,0.96016,0.90917,0.943439,-0.957003,-0.873877,-0.915361,0.785167
38091.0,-0.745956,-0.797635,0.817271,-0.845858,-0.749923,-0.82552,-0.865476,0.811402,0.804101,-0.836534,...,-0.537747,0.730117,-0.59308,-0.806767,-0.714099,-0.785672,-0.695805,-0.779958,0.717276,0.597079
38093.0,-0.995514,-0.971415,-0.954214,1.010596,-0.965519,0.99984,1.043314,0.992618,0.922329,1.009866,...,-0.787061,-0.942432,-0.87381,0.946593,0.876917,0.931127,-0.970682,-0.908517,-0.956885,-0.921104
38095.0,-0.844436,-0.876489,-0.842197,-0.929037,-0.868217,-0.879633,-0.91496,0.852063,-0.843727,-0.893995,...,-0.595075,-0.784437,-0.661103,-0.795655,-0.76615,-0.832697,-0.75987,-0.776236,-0.728374,-0.642649
